In [ ]:
from google.colab import drive 
drive.mount('/mntDrive')

Mounted at /mntDrive


In [ ]:
! rm -r ocrpostcorrection

In [ ]:
!git clone https://github.com/jvdzwaan/ocrpostcorrection.git

Cloning into 'ocrpostcorrection'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 759 (delta 116), reused 108 (delta 57), pack-reused 588
Receiving objects: 100% (759/759), 1.18 MiB | 18.08 MiB/s, done.
Resolving deltas: 100% (481/481), done.


In [ ]:
!pip install ./ocrpostcorrection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ocrpostcorrection
  Preparing metadata (setup.py) ... done
  Created wheel for ocrpostcorrection: filename=ocrpostcorrection-0.0.1-py3-none-any.whl size=25015 sha256=029dcb33d0c464f2b2a9c30c6fdc23eeaafda409e0d360964bd4eb03345ba531
  Stored in directory: /tmp/pip-ephem-wheel-cache-2g6zrole/wheels/b3/70/e7/fb04ee78a8446af5b0ef98bff0ddc7329cbac9364ab86bc8b2
Successfully built ocrpostcorrection
  Attempting uninstall: ocrpostcorrection
    Found existing installation: ocrpostcorrection 0.0.1
    Uninstalling ocrpostcorrection-0.0.1:
      Successfully uninstalled ocrpostcorrection-0.0.1


In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading data

In [ ]:
data_base_dir = Path('/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ocrpostcorrection')

In [ ]:
data_base_dir = Path('/mntDrive/MyDrive/data/ocrpostcorrection')

### Dataset without duplicates

In [ ]:
in_file = data_base_dir/'icdar-task2-dataset-20221031'/'task2dataset-no-duplicates.csv'
data = pd.read_csv(in_file, index_col=0)
data = data.fillna('')

In [ ]:
train = data.query('dataset == "train"')
test = data.query('dataset == "test"')

### Test set from 'perfect' ICDAR output for error detection 

In [ ]:
# Local
test_data_dir = Path('/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')

In [ ]:
# Google Drive
test_data_dir = data_base_dir / 'ICDAR2019_POCR_competition_dataset' / 'ICDAR2019_POCR_competition_evaluation_4M_without_Finnish'

In [ ]:
from ocrpostcorrection.icdar_data import generate_data
from ocrpostcorrection.utils import create_perfect_icdar_output, icdar_output2simple_correction_dataset_df

data_test, md_test = generate_data(test_data_dir)

output = create_perfect_icdar_output(data_test)

test = icdar_output2simple_correction_dataset_df(output, data_test)

10it [08:37, 51.73s/it]


In [ ]:
from ocrpostcorrection.error_correction import generate_vocabs, get_text_transform

vocab_transform = generate_vocabs(train)
text_transform = get_text_transform(vocab_transform)

In [ ]:
from torch.utils.data import DataLoader

from ocrpostcorrection.error_correction import SimpleCorrectionDataset, collate_fn

max_len = 22
batch_size = 256

test_dataset = SimpleCorrectionDataset(test, max_len=max_len)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn(text_transform))

## Load model

In [ ]:
from ocrpostcorrection.error_correction import SimpleCorrectionSeq2seq

hidden_size = 256
dropout = 0.1
model = SimpleCorrectionSeq2seq(len(vocab_transform['ocr']), 
                                hidden_size, 
                                len(vocab_transform['gs']), 
                                dropout, 
                                max_len, 
                                teacher_forcing_ratio=0.5,
                                device=device)
model.to(device)    
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
!ls /mntDrive/MyDrive/data

ocrpostcorrection


In [ ]:
model_save_path = data_base_dir/'results'/'simple_correction_model_2023-01-14'/'model.rar'

checkpoint = torch.load(model_save_path, map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        
model = model.to(device)

In [ ]:
from ocrpostcorrection.error_correction import validate_model

validate_model(model, test_dataloader, device)

7.2396957493332295

In [ ]:
from ocrpostcorrection.error_correction import predict_and_convert_to_str

predictions = predict_and_convert_to_str(model, test_dataloader, vocab_transform['gs'], device)

100%|██████████| 1486/1486 [06:12<00:00,  3.99it/s]


In [ ]:
test_results = test.query(f'len_ocr <= {max_len}').query(f'len_gs <= {max_len}').copy()

test_results['pred'] = predictions

In [ ]:
import edlib 

test_results['ed'] = test_results.apply(lambda row: edlib.align(row.ocr, row.gs)['editDistance'], axis=1)
test_results.ed.describe()

count    380390.000000
mean          2.475927
std           2.043403
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          22.000000
Name: ed, dtype: float64

In [ ]:
from ocrpostcorrection.icdar_data import normalized_ed

test_results['ed_norm'] = test_results.apply(lambda row: normalized_ed(row.ed, row.ocr, row.gs), axis=1)
test_results.ed_norm.describe()

count    380390.000000
mean          0.373365
std           0.225684
min           0.045455
25%           0.200000
50%           0.333333
75%           0.500000
max           1.000000
Name: ed_norm, dtype: float64

In [ ]:
import edlib 

test_results['ed_pred'] = test_results.apply(lambda row: edlib.align(row.pred, row.gs)['editDistance'], axis=1)
test_results.ed_pred.describe()

count    380390.000000
mean          1.581611
std           2.282115
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max          23.000000
Name: ed_pred, dtype: float64

In [ ]:
import pandas as pd

pd.DataFrame([test_results.ed.describe(), test_results.ed_pred.describe()])

,count,mean,std,min,25%,50%,75%,max
ed,380390.0,2.475927,2.043403,1.0,1.0,2.0,3.0,22.0
ed_pred,380390.0,1.581611,2.282115,0.0,0.0,1.0,2.0,23.0


In [ ]:
test_results['ed_norm_pred'] = test_results.apply(lambda row: normalized_ed(row.ed_pred, row.pred, row.gs), axis=1)
test_results.ed_norm_pred.describe()

count    380390.000000
mean          0.220827
std           0.269454
min           0.000000
25%           0.000000
50%           0.142857
75%           0.333333
max           1.000000
Name: ed_norm_pred, dtype: float64

In [ ]:
(test_results.pred == test_results.gs).sum()/test_results.shape[0]

0.42498751281579433

In [ ]:
test_results[test_results.pred == test_results.gs].sample(5)

,ocr,gs,start,text,len_ocr,len_gs,language,subset,dataset,pred,ed,ed_norm,ed_pred,ed_norm_pred
136595,cin,ein,324,DE/DE3/358.txt,3,3,DE,DE3,test,ein,1,0.333333,0,0.0
301355,obne,ohne,724,DE/DE3/273.txt,4,4,DE,DE3,test,ohne,1,0.250000,0,0.0
372189,nécessaire,necessaire,736,FR/FR2/48.txt,10,10,FR,FR2,test,necessaire,1,0.100000,0,0.0
276702,bdie,die,1059,DE/DE3/1497.txt,4,3,DE,DE3,test,die,1,0.250000,0,0.0
187621,Gtaatcé6.,Staates.,1434,DE/DE3/1016.txt,9,8,DE,DE3,test,Staates.,4,0.444444,0,0.0


In [ ]:
out_file = data_base_dir/'results'/'simple_correction_model'/'predictions.csv'
test_results.to_csv(out_file)

In [ ]:
out_file = data_base_dir/'results'/'simple_correction_model'/'predictions_test_data.csv'
test_results.to_csv(out_file)